In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

import json

In [ ]:
# a small little ruse
headers = {
    'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
}

In [ ]:
#test urls
url_1 = 'https://www.mlb.com/schedule/2022-09-29'
url_2 = 'https://www.mlb.com/schedule/2022-10-02'
url_3 = 'https://www.mlb.com/schedule/2022-09-11'
url_4 = 'https://www.mlb.com/schedule/2022-10-05'
url_5 ='https://www.mlb.com/schedule/2022-10-08' # this one does not have reg season games

In [ ]:
# need to get dates for first game of 2017 season 2017


# stop at last regular season game for 2019

# skip 2020
# need to get date for start of 2021

# stop at last regular season game

In [ ]:
def to_a_df(url):
    
    day_of_week = []
    months = []
    dates = []
    away_teams = []
    home_teams = []
    winning_teams = []
    winning_scores = []
    losing_teams = []
    losing_scores = []
    
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    # find the table from website
    table = soup.find('section', class_ = 'SnSstyle__GameCardsWrapper-sc-1m2zl7j-0 kvgDVy')
    table

    string = table.findChildren()[0].text
    
     
    DAY_COUNT = 0

    first_pass = True

    while DAY_COUNT < 3: 

        if first_pass: 
                
            split_first_space = string.split(' ',1)  

            # split_first_space[0] = 'WednesdayOct'
            day = split_first_space[0][:-3]     # 'Wednesday' 'Thursday' 'Friday'
            month = split_first_space[0][-3:]  # 'Oct' 'Oct' 'Oct'

            split_first = split_first_space[1].split(' ', 1)  

            # postponed games are always listed last (no played games will come after so skip
            # to new day and start loop over)
            if 'Postponed' in split_first[0]:
            
                # add to DAY_COUNT
                DAY_COUNT += 1

                postpone_split = split_first[0].split('Postponed')
                string = postpone_split[1]+' '+split_first[1]         

            # if there is an '@' within first 10 characters then there is a game on this day
            elif '@' in split_first[0][:10]: 

                # split first element at the '@' symbol, the number is the DATE 
                split_at = split_first[0].split('@')

                day_of_week.append(day)
                months.append(month)
            
                date = ''.join(filter(str.isdigit, split_at[0]))  # '5' is the date because this is first pass
                dates.append(date) 

                almost_away = split_at[0].replace(date, '')
                mirror_point = int(len(almost_away)/2)
                away = almost_away[:mirror_point]
                away_teams.append(away)


                if int(len(split_at[1])) == 9 or 8:
                    home = split_at[1][:3]   # 'BAL'
                elif int(len(split_at[1])) == 6 or 7:
                    home = split_at[1][:2]
                home_teams.append(home)

                if split_at[1] == home+home+home:
                    win = home
                elif split_at[1] == away+away+away:
                    win = away
                elif split_at[1] == home+home+away:
                    win = away
                else: #split_at[1] == away+away+home
                    win = home
                winning_teams.append(win)
                
                split_comma = split_first[1].split(', ', 1)  

                W_score = split_comma[0] # '5'
                winning_scores.append(W_score)

                split_space = split_comma[1].split(' ', 1)

                lose = split_space[0]  # 'TOR'
                losing_teams.append(lose)

                split_first = split_space[1].split(' ', 1) 

                first_pass = False

            else:
            # if there is NOT an '@' in first 10 characters of split_first[0] then there are NO GAMES on this day
            # there will be a message that says no games -or- says it is not a regular season game:

                # add to DAY_COUNT
                DAY_COUNT += 1
                
                no_game_sentence = 'There are no games scheduled for this date.'

                wild_card_sentence = 'Card'

                series = 'Series'


                 # How to start loop over
                if no_game_sentence in split_first_space[1][:50]:

                    resetting_string = split_first_space[1].split(no_game_sentence, 1)

                    string = resetting_string[1]

                else:

                    # look for next day of the week and start there
                    # NEED TO FIND NEXT DAY
                    for dow in ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']:

                        if dow in split_first_space[1]:

                            find_this_day_of_week = dow

                            break # out of day of the week for loop

                    start_day_of_week_substring = split_first_space[1].find(dow)

                    if start_day_of_week_substring != -1:
                        string = split_first_space[1][start_day_of_week_substring:]
                        
        else: # if first_pass = False

            L_score = ''.join(filter(str.isdigit, split_first[0]))  # '4' losing score for first day
            losing_scores.append(L_score) 

            # postponed games are always listed last (no played games will come after so skip
            # to new day and start loop over)
            if 'Postponed' in split_first[0]:
                
                # add to DAY_COUNT
                DAY_COUNT += 1
                
                first_pass = True

                postpone_split = split_first[0].split('Postponed')
               
                if len(split_first) > 1:
                    string = postpone_split[1]+' '+split_first[1]
                    
                     
            # if there is an '@' within first 10 characters then there is a game on this day
            elif '@' in split_first[0][:10]: 

                # split first element at the '@' symbol, the number is the DATE 
                split_at = split_first[0].split('@')

                # new game starts here
                day_of_week.append(day)
                months.append(month) 
                dates.append(date)

                almost_away = split_at[0].replace(L_score, '')
                mirror_point = int(len(almost_away)/2)
                away = almost_away[:mirror_point]
                away_teams.append(away)

                
                if (int(len(split_at[1])) == 9) or (int(len(split_at[1])) == 8):
                    home = split_at[1][:3]   # 'BAL'
                else:
                    home = split_at[1][:2]
                home_teams.append(home)

                
                if split_at[1] == home+home+home:
                    win = home
                elif split_at[1] == away+away+away:
                    win = away
                elif split_at[1] == home+home+away:
                    win = away
                else: #split_at[1] == away+away+home
                    win = home
                winning_teams.append(win)

                split_comma = split_first[1].split(', ', 1)  

                W_score = split_comma[0] # '5'
                winning_scores.append(W_score)
               
                split_space = split_comma[1].split(' ', 1)

                lose = split_space[0]  # 'BAL'
                losing_teams.append(lose)
                
                split_first = split_space[1].split(' ', 1) 

            else:
                # if there is not an '@' within the first 10 characters of split_first[0]
                remove_num = split_first[0].replace(L_score, '') # example: 'ThursdayOct'
                DAY_COUNT += 1
                first_pass = True

                if len(split_first) > 1:
                    string = remove_num+' '+split_first[1]  # example: 'ThursdayOct 6There are...'

                    
                    
    to_df_dict = {'Day': day_of_week, 
                  'Month': months, 
                  'Date': dates, 
                  'Away': away_teams, 
                  'Home': home_teams, 
                  'Win': winning_teams, 
                  'W Score': winning_scores, 
                  'Lose': losing_teams, 
                  'L Score': losing_scores}

    return pd.DataFrame(data=to_df_dict)

In [1]:
# keep the below for trouble shooting if needed

In [ ]:
"""
url = 'https://www.mlb.com/schedule/2022-09-29'

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

# find the table from website
table = soup.find('section', class_ = 'SnSstyle__GameCardsWrapper-sc-1m2zl7j-0 kvgDVy')
table

info_2 = table.findChildren()[0].text
info_2
"""
print()

In [ ]:
"""
url = 'https://www.mlb.com/schedule/2022-10-02'

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

# find the table from website
table = soup.find('section', class_ = 'SnSstyle__GameCardsWrapper-sc-1m2zl7j-0 kvgDVy')
table

info_3 = table.findChildren()[0].text
info_3
"""
print()

In [ ]:
"""
url = 'https://www.mlb.com/schedule/2022-09-11'

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

# find the table from website
table = soup.find('section', class_ = 'SnSstyle__GameCardsWrapper-sc-1m2zl7j-0 kvgDVy')
table

info_4 = table.findChildren()[0].text
info_4
"""
print()

In [ ]:
# keep for trouble shooting

# this is the REAL copy to mess up
# get to the container with the data from games by day
info = info_4


day_of_week = []
months = []
dates = []
away_teams = []
home_teams = []
winning_teams = []
winning_scores = []
losing_teams = []
losing_scores = []


    ### GAME 1 IN THIS FIRST SECTION IT NEEDS TO RECOGNINZE NOT GAMES STRING AND GO TO NENXT DAY

DAY_COUNT = 0

first_pass = True

while DAY_COUNT < 3: 
    
    
    if first_pass: 
        print(f'this is day count: {DAY_COUNT}')
        print(f'this is the first pass: first_pass = {first_pass}')
        
        split_first_space = info.split(' ',1)  
        
        # split_first_space[0] = 'WednesdayOct'
        day = split_first_space[0][:-3]     # 'Wednesday' 'Thursday' 'Friday'
        month = split_first_space[0][-3:]  # 'Oct' 'Oct' 'Oct'
                                                                                            
        split_first = split_first_space[1].split(' ', 1)  
           
        # postponed games are always listed last (no played games will come after so skip
        # to new day and start loop over)
        if 'Postponed' in split_first[0]:
            print(f'this is the first loop: split_first[0] = {split_first[0]} and it has Postponed in it')

            # add to DAY_COUNT
            DAY_COUNT += 1
            print(f'I am adding 1 to the day count, now DAY_COUNT = {DAY_COUNT}')

            postpone_split = split_first[0].split('Postponed')
            info = postpone_split[1]+' '+split_first[1]
            print('I have re set info to:')
            print(info)
            print()
            print('the loop will now start over.')
            


        # if there is an '@' within first 10 characters then there is a game on this day
        elif '@' in split_first[0][:10]: 
            print(f"""this is the first loop: split_first[0] = {split_first[0]} and
            it does have an @ sign AND it does not have 'Postponed' in it so I need to 
            collect game data.""")
            print()
                
            # split first element at the '@' symbol, the number is the DATE 
            split_at = split_first[0].split('@')
            
            day_of_week.append(day)
            months.append(month)
            print(f'\tday: {day}')
            print(f'\tmonth: {month}')
            
            date = ''.join(filter(str.isdigit, split_at[0]))  # '5' is the date because this is first pass
            dates.append(date) 
            print(f'\tdate: {date}')
            
            #away = split_at[0][-3:]  # 'TOR'
            almost_away = split_at[0].replace(date, '')
            mirror_point = int(len(almost_away)/2)
            away = almost_away[:mirror_point]
            
            away_teams.append(away)
            print(f'\tthe away team is: {away}')
                
                
            if int(len(split_at[1])) == 9 or 8:
                home = split_at[1][:3]   # 'BAL'
            elif int(len(split_at[1])) == 6 or 7:
                home = split_at[1][:2]
            home_teams.append(home)
            print(f'\tthe home team is: {home}')
            
            if split_at[1] == home+home+home:
                win = home
            elif split_at[1] == away+away+away:
                win = away
            elif split_at[1] == home+home+away:
                win = away
            else: #split_at[1] == away+away+home
                win = home
            
            winning_teams.append(win)
            print(f'\tthe winning team is: {win}')
            
            
            split_comma = split_first[1].split(', ', 1)  
            

            W_score = split_comma[0] # '5'
            winning_scores.append(W_score)
            print(f'\tthe winning team scored: {W_score} runs')
            
            split_space = split_comma[1].split(' ', 1)

            lose = split_space[0]  # 'TOR'
            losing_teams.append(lose)
            print(f'\tthe losing team is: {lose}')

            split_first = split_space[1].split(' ', 1) 
            
            first_pass = False
            print(f'I am changing first pass, now first_pass = {first_pass}')

            ####### is there where loop starts over but 4 is l_score not date?
            

        
        else:
            print(f"""this is the first loop: split_first[0] = {split_first[0]} 
and it does not have an @ sign. Because first_pass = {first_pass}, there
are no games on this day, or they are not regular season games. I need to skip
to the NEXT DAY.""")
            print()
        # if there is NOT an '@' in first 10 characters of split_first[0] then there are NO GAMES on this day
        # there will be a message that says no games -or- says it is not a regular season game:
            
            # add to DAY_COUNT
            DAY_COUNT += 1
            print(f'I have added 1 to the day count, it is now: {DAY_COUNT}')
            print()
            
            no_game_sentence = 'There are no games scheduled for this date.'
            
            wild_card_sentence = 'Card'
            
            series = 'Series'
            
            
             # How to start loop over
            if no_game_sentence in split_first_space[1][:50]:
                
                resetting_info = split_first_space[1].split(no_game_sentence, 1)
                
                info = resetting_info[1]
                print(f'I have updated info. It is now:')
                print(info)
                print()
                
                        
            else:
                
                # look for next day of the week and start there
                # NEED TO FIND NEXT DAY
                for dow in ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']:

                    if dow in split_first_space[1]:

                        find_this_day_of_week = dow
                        
                        break # out of day of the week for loop
                        
                start_day_of_week_substring = split_first_space[1].find(dow)
                
                if start_day_of_week_substring != -1:
                    info = split_first_space[1][start_day_of_week_substring:]
                    print(f'I have updated info. It is now:')
                    print(info)
                    print()

                
    
                    
    else: # if first_pass = False
        
        print(f"first_pass = {first_pass} so I need to continue collecting game data.")
        
        L_score = ''.join(filter(str.isdigit, split_first[0]))  # '4' losing score for first day
        losing_scores.append(L_score) 

        # postponed games are always listed last (no played games will come after so skip
        # to new day and start loop over)
        if 'Postponed' in split_first[0]:
            print(f"""this is the second loop: split_first[0] = {split_first[0]} and
            it does have 'Postponed' in it, so I need to skip to the next day.""")
            print()
            
          #  losing_scores.append(L_score)
            print(f'why do i think this is where I append losing score?')
            print(f'L_score = {L_score}')
            print()

            # add to DAY_COUNT
            DAY_COUNT += 1
            print(f'I have added to the day count, day count is now {DAY_COUNT}')

            postpone_split = split_first[0].split('Postponed')
            print()
            print(f'this is the second loop: postpone_split = {postpone_split}')

            if len(split_first) > 1:
                
                info = postpone_split[1]+' '+split_first[1]
                print(f'Time to move to NEXT DAY. I have updated info to:')
                print()
                print(info)
                print()

            
            first_pass = True
            print(f'I have changed first pass to True, first_pass = {first_pass}')


        # if there is an '@' within first 10 characters then there is a game on this day
        elif '@' in split_first[0][:10]: 
            print(f"""this is the second loop: split_first[0] = {split_first[0]} and
            it does have an @ sign AND it does not have 'Postponed' in it, so I need to 
            collect game data. Because I am in the second loop I need to get the 
            prior games losing score before starting a new game.""")
            print()
               
            # split first element at the '@' symbol, the number is the DATE 
            split_at = split_first[0].split('@')
            
            # append the losing score
     #       losing_scores.append(L_score)   ########################
            print(f'\tthe losing team scored {L_score} runs')
            print()
            
            # new game starts here
            day_of_week.append(day)
            months.append(month) 
            dates.append(date)
            print(f'THIS IS A NEW GAME ON THE SAME DAY')
            print(f'This is still DAY_COUNT: {DAY_COUNT}')
            
            
            almost_away = split_at[0].replace(L_score, '')
            mirror_point = int(len(almost_away)/2)
            away = almost_away[:mirror_point]
            
            away_teams.append(away)
            print(f'\tthe away team is: {away}')
                
            if (int(len(split_at[1])) == 9) or (int(len(split_at[1])) == 8):
                home = split_at[1][:3]   # 'BAL'
            else:
                home = split_at[1][:2]
                
            home_teams.append(home)
            print(f'\tthe home team is: {home}')
            
            if split_at[1] == home+home+home:
                win = home
            elif split_at[1] == away+away+away:
                win = away
            elif split_at[1] == home+home+away:
                win = away
            else: #split_at[1] == away+away+home
                win = home
            
            winning_teams.append(win)
            print(f'\tthe winning team is: {win}')
            
            split_comma = split_first[1].split(', ', 1)  

            W_score = split_comma[0] # '5'
            winning_scores.append(W_score)
            print(f'\tthe winning team scored: {W_score} runs')

            split_space = split_comma[1].split(' ', 1)

            lose = split_space[0]  # 'BAL'
            losing_teams.append(lose)
            print(f'\tthe losing team is: {lose}')

            split_first = split_space[1].split(' ', 1) 
            
        else:
            # if there is not an '@' within the first 10 characters of split_first[0]
            print(f"""this is the second loop: split_first[0] = {split_first[0]} and
            it does NOT have an @ sign so this is the start of a new day. But first
            I need to get the losing score of the proir game.""")
            print()
           
            # append the losing score
   #         losing_scores.append(L_score)   ########################
            print(f'\tthe losing team scored {L_score} runs')
            print()
            
            remove_num = split_first[0].replace(L_score, '') # example: 'ThursdayOct'
            DAY_COUNT += 1
            first_pass = True
            
            if len(split_first) > 1:
                info = remove_num+' '+split_first[1]  # example: 'ThursdayOct 6There are...'
                
            
                print(f'Time to move to NEXT DAY. I have updated info to:')
                print()
                print(info)
                print()
            print(f'I have added to the day count, day count is now {DAY_COUNT}')
            print(f'I have changed first pass to True, first_pass = {first_pass}')
            
            
            


to_df_dict = {'Day': day_of_week, 
              'Month': months, 
              'Date': dates, 
              'Away': away_teams, 
              'Home': home_teams, 
              'Win': winning_teams, 
              'W Score': winning_scores, 
              'Lose': losing_teams, 
              'L Score': losing_scores}

pd.DataFrame(data=to_df_dict)